In [10]:
%load_ext autoreload
%autoreload 1
%aimport src
%aimport src.utils
%aimport src.SdmIsmAnalysis
%aimport src.SimpleSynthesis

# display last assigned variable automatically
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr_or_assign"



%matplotlib inline
from matplotlib.pyplot import *

import polars as pl

from src.utils import mygrid
from pathlib import Path

import pyperclip

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df = pl.read_parquet("listening_test_design.parquet")

index,to_listener,to_source,room,from_listener_good,from_source_good,from_listener_bad,from_source_bad,az,az_idx
u32,str,i32,str,str,i32,str,i32,i32,u32
0,"""0.0X_0.0Y""",1,"""HL05W""","""1.75X_-2.0Y""",1,"""3.0X_2.0Y""",1,0,6
1,"""3.0X_2.0Y""",4,"""HL06W""","""3.0X_2.0Y""",1,"""3.0X_2.0Y""",3,-150,1
2,"""1.75X_-2.0Y""",2,"""HL05W""","""1.75X_-2.0Y""",0,"""3.0X_2.0Y""",1,-30,5
3,"""1.75X_-2.0Y""",0,"""HL06W""","""3.0X_2.0Y""",0,"""3.0X_2.0Y""",3,120,10
4,"""1.75X_-2.0Y""",1,"""HL05W""","""1.75X_-2.0Y""",0,"""3.0X_2.0Y""",3,60,8
…,…,…,…,…,…,…,…,…,…
15,"""-1.25X_0.0Y""",3,"""HL06W""","""3.0X_2.0Y""",0,"""3.0X_2.0Y""",4,-60,4
16,"""1.75X_-2.0Y""",2,"""HL06W""","""3.0X_2.0Y""",0,"""0.0X_0.0Y""",4,-30,5
17,"""3.0X_2.0Y""",3,"""HL05W""","""0.0X_0.0Y""",3,"""1.75X_-2.0Y""",4,-90,3


In [4]:
df.columns

['index',
 'to_listener',
 'to_source',
 'room',
 'from_listener_good',
 'from_source_good',
 'from_listener_bad',
 'from_source_bad',
 'az',
 'az_idx']

In [16]:
df_sel = df.select([
    'index',
    'room',
    'from_listener_bad',
    'from_source_bad',
    'to_listener',
    'to_source',
    'az',
]).rename(
    dict(
        index="trial",
        from_listener_bad="from_listener",
        from_source_bad="from_source",
        az="azimuth",
    )
)

trial,room,from_listener,from_source,to_listener,to_source,azimuth
u32,str,str,i32,str,i32,i32
0,"""HL05W""","""3.0X_2.0Y""",1,"""0.0X_0.0Y""",1,0
1,"""HL06W""","""3.0X_2.0Y""",3,"""3.0X_2.0Y""",4,-150
2,"""HL05W""","""3.0X_2.0Y""",1,"""1.75X_-2.0Y""",2,-30
3,"""HL06W""","""3.0X_2.0Y""",3,"""1.75X_-2.0Y""",0,120
4,"""HL05W""","""3.0X_2.0Y""",3,"""1.75X_-2.0Y""",1,60
…,…,…,…,…,…,…
15,"""HL06W""","""3.0X_2.0Y""",4,"""-1.25X_0.0Y""",3,-60
16,"""HL06W""","""0.0X_0.0Y""",4,"""1.75X_-2.0Y""",2,-30
17,"""HL05W""","""1.75X_-2.0Y""",4,"""3.0X_2.0Y""",3,-90


In [18]:
trials_exp_order = [14, 11, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 12, 15, 16, 17, 18, 19]

[14, 11, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 12, 15, 16, 17, 18, 19]

In [19]:
pyperclip.copy(df_sel[trials_exp_order].to_pandas().to_latex(escape=True, index=False))